In [ ]:
%pip install langchain-groq

In [5]:
from typing import List
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

In [11]:
re_write_llm = ChatGroq(temperature=0, model="llama-3.1-70b-versatile", max_tokens=4000,api_key='')

In [12]:
# Create a prompt template for query rewriting
query_rewrite_template = """You are an AI assistant tasked with reformulating user queries to improve retrieval in a RAG system. 
Given the original query, rewrite it to be more specific, detailed, and likely to retrieve relevant information.

Original query: {original_query}

Rewritten query:"""

query_rewrite_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=query_rewrite_template
)


In [13]:
query_rewriter = query_rewrite_prompt | re_write_llm

In [14]:
def rewrite_query(original_query):
    """
    Rewrite the original query to improve retrieval.
    
    Args:
    original_query (str): The original user query
    
    Returns:
    str: The rewritten query
    """
    response = query_rewriter.invoke(original_query)
    return response.content

In [15]:
# example query over the understanding climate change dataset
original_query = "What are the impacts of climate change on the environment?"
rewritten_query = rewrite_query(original_query)
print("Original query:", original_query)
print("\nRewritten query:", rewritten_query)

Original query: What are the impacts of climate change on the environment?

Rewritten query: Rewritten query: What are the specific effects of rising global temperatures, sea-level rise, and altered precipitation patterns on ecosystems, biodiversity, and natural resources, including changes in weather patterns, water scarcity, and loss of endangered species, and how do these impacts vary across different regions and ecosystems, such as coral reefs, polar ice caps, and forests?


In [16]:
step_back_llm = ChatGroq(temperature=0, model="llama-3.1-70b-versatile", max_tokens=4000,api_key='')

In [17]:
step_back_template = """You are an AI assistant tasked with generating broader, more general queries to improve context retrieval in a RAG system.
Given the original query, generate a step-back query that is more general and can help retrieve relevant background information.

Original query: {original_query}

Step-back query:"""

step_back_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=step_back_template
)

In [18]:
step_back_chain = step_back_prompt | step_back_llm

def generate_step_back_query(original_query):
    """
    Generate a step-back query to retrieve broader context.
    
    Args:
    original_query (str): The original user query
    
    Returns:
    str: The step-back query
    """
    response = step_back_chain.invoke(original_query)
    return response.content

In [19]:
original_query = "What are the impacts of climate change on the environment?"
step_back_query = generate_step_back_query(original_query)
print("Original query:", original_query)
print("\nStep-back query:", step_back_query)

Original query: What are the impacts of climate change on the environment?

Step-back query: Step-back query: What is climate change and how does it affect the planet? 

(This step-back query is more general and can help retrieve relevant background information about climate change, its causes, and its effects, which can provide a broader context for understanding its impacts on the environment.)


In [20]:
sub_query_llm = ChatGroq(temperature=0, model="llama-3.1-70b-versatile", max_tokens=4000,api_key='')

In [21]:
# Create a prompt template for sub-query decomposition
subquery_decomposition_template = """You are an AI assistant tasked with breaking down complex queries into simpler sub-queries for a RAG system.
Given the original query, decompose it into 2-4 simpler sub-queries that, when answered together, would provide a comprehensive response to the original query.

Original query: {original_query}

example: What are the impacts of climate change on the environment?

Sub-queries:
1. What are the impacts of climate change on biodiversity?
2. How does climate change affect the oceans?
3. What are the effects of climate change on agriculture?
4. What are the impacts of climate change on human health?"""

In [22]:
subquery_decomposition_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=subquery_decomposition_template
)

# Create an LLMChain for sub-query decomposition
subquery_decomposer_chain = subquery_decomposition_prompt | sub_query_llm

In [24]:
def decompose_query(original_query: str):
    """
    Decompose the original query into simpler sub-queries.
    
    Args:
    original_query (str): The original complex query
    
    Returns:
    List[str]: A list of simpler sub-queries
    """
    response = subquery_decomposer_chain.invoke(original_query).content
    return response

In [25]:
original_query = "What are the impacts of climate change on the environment?"
sub_queries = decompose_query(original_query)

In [27]:
sub_queries = [sub_query.strip() for sub_query in sub_queries.split("\n") if sub_query.strip()]

In [28]:
sub_queries

["To break down the original query into simpler sub-queries, I'll consider the various aspects of the environment that climate change can impact. Here are the sub-queries:",
 'Original query: What are the impacts of climate change on the environment?',
 'Sub-queries:',
 '1. How does climate change affect global temperature and weather patterns?',
 '2. What are the impacts of climate change on ecosystems, including deforestation and wildlife habitats?',
 '3. How does climate change influence water resources, including sea-level rise, glaciers, and freshwater availability?',
 '4. What are the effects of climate change on soil quality, air pollution, and overall environmental degradation?',
 'These sub-queries cover various aspects of the environment, including atmospheric, terrestrial, and hydrological systems. Answering these sub-queries together will provide a comprehensive understanding of the impacts of climate change on the environment.']